In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
rocket = sns.color_palette("rocket", as_cmap=True)

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

from tqdm.notebook import tqdm
from toric import *
# from mwpm import *

In [ ]:
# Simulation parameters

L = 50 # Lattice size
p_error = 0.06 # Error probability per spin
η = 0.5 # Smoothing paramter for Jacobi method
c = 10 # "Field velocity" - number of field updates per cycle
T = 100 # Epochs

In [ ]:
mystate = init_state(L, p_error)
q_history, error_history = decoder_2D(mystate, T, c, η, history = True)
print(mystate.N > 0 or logical_error(mystate.error))
plot_evolution(q_history, error_history, trail = False)

In [ ]:
fail_rate = np.ones((60, 60)) # L/2 - 1, c - 1
shots = 100 # error per pixel is max. +/- 0.05

In [ ]:
for c in tqdm(range(1, 61)):
    for L in range(2, 122, 2):
        T = 10 * L
        fails = 0
        for _ in range(shots):
            mystate = init_state(L, p_error)
            decoder_2D(mystate, T, c, η, history = False)
            fails += (mystate.N > 0 or logical_error(mystate.error))
        fail_rate[L // 2 - 1, c - 1] = fails / shots

np.save("fail_rate.npy", fail_rate)

In [ ]:
from matplotlib.colors import LogNorm
fig, ax = plt.subplots()
mat = ax.matshow(fail_rate.T, origin = 'lower', cmap = "viridis_r", norm = LogNorm())
fig.colorbar(mat)
ax.set_xlabel('System size $L/2-1$')
ax.set_ylabel('Field velocity $c-1$')
ax.xaxis.tick_bottom()
ax.set_title('Failure rate')
plt.savefig('fail_rate.png')
plt.show()